### import

In [2]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import optuna
import lightgbm as lgb

### Fixed RandomSeed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### Data Load

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,17.55


### Split

In [5]:
train_x = train.drop(['ID', '가격'], axis = 1)
train_y = train['가격']

test_x = test.drop('ID', axis = 1)

### Data Processing

In [6]:
ordinal_features = ['브랜드', '차량모델명', '판매도시', '판매구역']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])

    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고유값을 확인후 test 데이터를 변환합니다.
    # Data Leakage를 발생시키지 않기 위함이니, 반드시 주의해주세요.
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])

### Train-Validation Split

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### LightGBM 데이터셋으로 변환

In [8]:
train_data = lgb.Dataset(X_train, label=y_train)

### Objective Function for Optuna

In [9]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mean_absolute_error',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 10, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'verbose': 0,
        'random_state': 42
    }

    # LightGBM 모델 학습
    model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data], early_stopping_rounds=10, verbose_eval=False)

    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val)

    # MAE 계산
    mae = mean_absolute_error(y_val, y_pred)
    return mae

### Optuna를 사용한 하이퍼파라미터 튜닝

In [10]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2023-06-13 02:10:02,848] A new study created in memory with name: no-name-36e2b59e-4e23-4697-97ff-b23b8984531c
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002812 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-06-13 02:10:03,680] Trial 0 finished with value: 7.005217065279506 and parameters: {'learning_rate': 0.23814700058746416, 'num_leaves': 82, 'feature_fraction': 0.6144363932068045, 'bagging_fraction': 0.15959120815834577, 'bagging_freq': 8}. Best is trial 0 with value: 7.005217065279506.
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a fu

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000239 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


[I 2023-06-13 02:10:04,355] Trial 1 finished with value: 6.639242525638719 and parameters: {'learning_rate': 0.09298980086357593, 'num_leaves': 38, 'feature_fraction': 0.5879477241609797, 'bagging_fraction': 0.8083029887432317, 'bagging_freq': 4}. Best is trial 1 with value: 6.639242525638719.
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a fut

...

[I 2023-06-13 02:11:42,683] Trial 97 finished with value: 6.0481086969374065 and parameters: {'learning_rate': 0.1949924508265502, 'num_leaves': 104, 'feature_fraction': 0.4708273189480902, 'bagging_fraction': 0.9844197540781021, 'bagging_freq': 4}. Best is trial 80 with value: 5.996730306488689.
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005334 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-06-13 02:11:43,636] Trial 98 finished with value: 6.076232882952512 and parameters: {'learning_rate': 0.24501124920407916, 'num_leaves': 84, 'feature_fraction': 0.41469225112044783, 'bagging_fraction': 0.8680469143573057, 'bagging_freq': 2}. Best is trial 80 with value: 5.996730306488689.
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jjaegii/dacon/venv/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003548 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-06-13 02:11:44,448] Trial 99 finished with value: 6.06537715647223 and parameters: {'learning_rate': 0.1777239223301375, 'num_leaves': 77, 'feature_fraction': 0.37358054475155206, 'bagging_fraction': 0.9556920414437668, 'bagging_freq': 2}. Best is trial 80 with value: 5.996730306488689.


### 최적의 하이퍼파라미터 출력

In [12]:
print("Best Hyperparameters:", study.best_params)

Best Hyperparameters: {'learning_rate': 0.196964601256542, 'num_leaves': 102, 'feature_fraction': 0.40679967020045177, 'bagging_fraction': 0.8801603093743194, 'bagging_freq': 3}


### 최적의 모델로 재학습

In [13]:
best_model = lgb.train({**study.best_params, 'verbose': 0}, train_data, num_boost_round=100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


### 검증 데이터에 대한 예측 수행 및 평가 지표 출력

In [14]:
y_pred = best_model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 6.061292161637324


### Inference & Submit

In [15]:
preds = best_model.predict(test_x)

In [16]:
submission = pd.read_csv('sample_submission.csv')
submission['가격'] = preds
submission.to_csv('./lightgbm_with_optuna_submit.csv', index = False)